# Viewing 2D projection and comparing with raw high dimensional values

There are many methods for visualizing high dimensional data in 2D. Some popular methods are PCA, t-SNE, autoencoding variants, amongst many others. These methods often do a great job at giving us a feel for what high dimensional data looks like; something our mere mortal brains can't do.

The main aim of this notebook is to setup a plotly dashboard that lets us interact with the 2D projection. This dashboard will consist of three panels:

- The left panel shows a scatter plot of the 2D projection
- The middle panel shows a line plot of the *raw high dimensional data* from a selected point in the 2D projection from the left panel. The idea here is we want to compare nearby points in the 2D embedding to see if they make sense in ther high dimensional counter-part.
- The right panel shows not a single line plot, but a collection of raw high dimensional data *points* from a group of selected points in the 2D projection from the left panel. The aim is similar to the middle panel but instead of focusing on a single data point we want to a look at a collection that we can selected using the box-select or lasso-select tool. 

In [1]:
import numpy as np
import pandas as pd

# plotly
from plotly.graph_objs import FigureWidget
from plotly.callbacks import Points, InputDeviceState

# ipywidgets
from ipywidgets import HTML
from ipywidgets import HBox, VBox, Button

## Data and projection setup

We load the data and prep and project to 2D. There are numerous methods for reducing the dimensionality of a dataset including PCA, t-SNE, autoencoders, and even random projections. Here we use PCA.

The data set that we use is the Iris data set (different types of flowers) that we can load directly from Scikit-learn:

In [2]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
X.shape

(150, 4)

`X` shows us that we have 150 different flowers where each flower is represented by a set of 4-dimensional featueres and `y` are the one-hot-encoded categories of different flower types.  


We will apply a PCA transformation with the top-2 component using the log-transformed data (often log-transformations on measurements will make the data more noraml distributed). 


In [3]:
from sklearn.decomposition import PCA

z = PCA(n_components=2).fit_transform(np.log10(X))

## Dashboard setup

Here is the magic of using plotly to make our dashbaord! First we will setup a figure for visualizing the projected data. 

In [4]:
## left panel figure
fig = FigureWidget(
    data=[
        dict(
            type='scattergl', # scattergl is ideal for scaling up visualization to over 10K points; 
            x=z[:,0], # the first latent (projeted) dimension
            y=z[:,1], # the second latent dimension
            mode='markers', # we want just dots in the scatter plot, by default they are connected
        )
    ],
)

# setup the color scale of the markers; 0 will correspond to lightgray;
# values over .5 will corresond to red and will mean selection when we make selections
fig.data[0].marker.cmax = 1.5
fig.data[0].marker.cmin = -.5
fig.data[0].marker.colorscale = [[0, 'lightgray'], [0.5, 'lightgray'],[0.5, 'red'], [1, 'red']]
fig.data[0].marker.color = np.zeros(X.shape[0])
# uncomment if you want a fixed width for the figure
#fig.layout.width = 500

# figure layout options that should be self explanatory
fig.layout.hovermode = "closest"
fig.layout.title = "Embedding"
fig.layout.xaxis.title = "z1"
fig.layout.yaxis.title = "z2"


# when we want to update the aesthetics of the figure based
# on data updates (such as selecting a subset of points), 
# we can specifically refer to `scatt1`
scatt1 = fig.data[0]

In [5]:
#
# Similarly we will setup the visualization of the raw features in 
# the middle figure. 
# 
fig2 = FigureWidget(
    data=[
        dict(
            type='scattergl',
            x= np.arange(X.shape[1]), # these are just the 4 features of flower
            y=X[0,:], # the value of the 4 features        
        )
    ],
)

#fig2.layout.width = 500
fig2.layout.hovermode = "closest"
fig2.layout.xaxis.title = "Features"
fig2.layout.title = "Selected sample"

In [6]:
#
# Finally we will setup the average set of features for a selected 
# group in the right figure. 
# 
fig3 = FigureWidget(
    data=[
        dict(
            type='scattergl',
            x= np.arange(X.shape[1]),
            y=np.mean(X,0),        
        )
    ],
)

#fig3.layout.width = 500
fig3.layout.hovermode = "closest"
fig3.layout.title = "Average values"
fig3.layout.xaxis.title = "Average values"

In [7]:
# Pandas has a nice method for displaying a table as an HTML, 
# so we will convert the data to a Pandas data frame 
df = pd.DataFrame(np.hstack((X, y[:,np.newaxis])), columns = ["feature " + str(i) for i in range(X.shape[1])] + ["flower type"])

In [8]:
# interactivity setup

selection_inds = []
def brush(trace, points, state):
    selection_inds.clear()
    selection_inds.extend(points.point_inds)  
    inds = np.array(points.point_inds)
    
    fig3.data[0].y = np.mean(X[points.point_inds,:],0)   
    
    
    
    selected_details.value = df.iloc[inds,:].to_html()
    
    for i in range(len(selection_inds)):
        fig3.add_scattergl(x = np.arange(X.shape[1]), y = X[selection_inds[i],:], 
                           marker = {"color": "lightblue"}, opacity = .3, showlegend=False)
    if inds.size:
        selected = scatt1.marker.color.copy()
        selected[inds] = 1
        scatt1.marker.color = selected 
        

    
scatt1.on_selection(brush)

def hover_fn(trace, points, state):
    ind = points.point_inds[0]
    fig2.data[0].y = X[ind,:]
    

scatt1.on_hover(hover_fn)


# Reset brush
def reset_brush(btn):
    selection_inds.clear()
    fig3.data = ()
    fig3.add_scattergl(x = np.arange(X.shape[1]), y = np.mean(X,0), 
                           showlegend=False)   
    scatt1.marker.color = np.zeros(X.shape[0])
    selected_details.value = ""

# Create reset button
button = Button(description="clear")
button.on_click(reset_brush)

In [9]:
dashboard = VBox([HBox([fig, fig2, fig3]), button])
dashboard

    'data': [{'marker': {'cmax': 1.5,
                         'c…

In [10]:
## the actual points we selected can be viewed here:
selected_details = HTML(width = 400)
selected_details

HTML(value='')